In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split


In [2]:
df_train_full=pd.read_csv('../input/train.csv')
print(len(df_train_full))

df_train_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
print(len(df_train_full))

1460
1460


In [3]:
y = df_train_full.SalePrice
df_train_full.drop(['SalePrice'], axis=1, inplace=True)
X_train, X_valid, y_train, y_valid = train_test_split(df_train_full, y, train_size=0.8, test_size=0.2,random_state=0)


In [4]:
print(len(X_train.index))
print(len(df_train_full.columns))
print(len(y_train))


1168
80
1168




**Taking care of null values**

  Those columns that have more than 75% missing values will be dropped


In [5]:
missing_val_count_by_column = (X_train.isnull().sum(axis=0))
# print(missing_val_count_by_column['Fence'])

cols_with_missing_values=np.array(missing_val_count_by_column[missing_val_count_by_column > 0].index)
print('Columns with missing values: {}'.format((cols_with_missing_values)))
 
cols_with_missing_more_than_75_percent =np.array(missing_val_count_by_column[missing_val_count_by_column > len(X_train.index)*(3/4)].index)
print('Columns with more than 75 percent missing values: {}'.format((cols_with_missing_more_than_75_percent)))

X_train=X_train.drop(cols_with_missing_more_than_75_percent,axis=1)
X_valid=X_valid.drop(cols_with_missing_more_than_75_percent,axis=1)

remaining_cols=np.setdiff1d(cols_with_missing_values,cols_with_missing_more_than_75_percent)


Columns with missing values: ['LotFrontage' 'Alley' 'MasVnrType' 'MasVnrArea' 'BsmtQual' 'BsmtCond'
 'BsmtExposure' 'BsmtFinType1' 'BsmtFinType2' 'Electrical' 'FireplaceQu'
 'GarageType' 'GarageYrBlt' 'GarageFinish' 'GarageQual' 'GarageCond'
 'PoolQC' 'Fence' 'MiscFeature']
Columns with more than 75 percent missing values: ['Alley' 'PoolQC' 'Fence' 'MiscFeature']


In [16]:
checking=(X_train.isnull().sum(axis=0))
print(checking)

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      212
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
Length: 76, dtype: int64


In [18]:

categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
print(len(categorical_cols))
print(len(numerical_cols))

39
37


In [20]:
for c in categorical_cols:
    print(checking[c],c)
print('------')
for c in numerical_cols:
    print(checking[c],c)  

0 MSZoning
0 Street
0 LotShape
0 LandContour
0 Utilities
0 LotConfig
0 LandSlope
0 Neighborhood
0 Condition1
0 Condition2
0 BldgType
0 HouseStyle
0 RoofStyle
0 RoofMatl
0 Exterior1st
0 Exterior2nd
6 MasVnrType
0 ExterQual
0 ExterCond
0 Foundation
28 BsmtQual
28 BsmtCond
28 BsmtExposure
28 BsmtFinType1
29 BsmtFinType2
0 Heating
0 HeatingQC
0 CentralAir
1 Electrical
0 KitchenQual
0 Functional
551 FireplaceQu
58 GarageType
58 GarageFinish
58 GarageQual
58 GarageCond
0 PavedDrive
0 SaleType
0 SaleCondition
------
0 Id
0 MSSubClass
212 LotFrontage
0 LotArea
0 OverallQual
0 OverallCond
0 YearBuilt
0 YearRemodAdd
6 MasVnrArea
0 BsmtFinSF1
0 BsmtFinSF2
0 BsmtUnfSF
0 TotalBsmtSF
0 1stFlrSF
0 2ndFlrSF
0 LowQualFinSF
0 GrLivArea
0 BsmtFullBath
0 BsmtHalfBath
0 FullBath
0 HalfBath
0 BedroomAbvGr
0 KitchenAbvGr
0 TotRmsAbvGrd
0 Fireplaces
58 GarageYrBlt
0 GarageCars
0 GarageArea
0 WoodDeckSF
0 OpenPorchSF
0 EnclosedPorch
0 3SsnPorch
0 ScreenPorch
0 PoolArea
0 MiscVal
0 MoSold
0 YrSold


In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [23]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05, n_jobs=4)




# my_model_1 = XGBRegressor(random_state=0) # Your code here
# model = RandomForestRegressor(n_estimators=100, criterion='mae',random_state=0)


pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', xgb_model)
                             ])

# Preprocessing of training data, fit model 
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'Ove

In [24]:
from sklearn.metrics import mean_absolute_error
preds = pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 15961.457539597603


In [25]:
test_df=pd.read_csv('../input/test.csv')


In [26]:
print(test_df)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0     1461          20       RH         80.0    11622   Pave   NaN      Reg   
1     1462          20       RL         81.0    14267   Pave   NaN      IR1   
2     1463          60       RL         74.0    13830   Pave   NaN      IR1   
3     1464          60       RL         78.0     9978   Pave   NaN      IR1   
4     1465         120       RL         43.0     5005   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1454  2915         160       RM         21.0     1936   Pave   NaN      Reg   
1455  2916         160       RM         21.0     1894   Pave   NaN      Reg   
1456  2917          20       RL        160.0    20000   Pave   NaN      Reg   
1457  2918          85       RL         62.0    10441   Pave   NaN      Reg   
1458  2919          60       RL         74.0     9627   Pave   NaN      Reg   

     LandContour Utilities  ... ScreenPorch PoolAre

In [37]:
X_test=test_df.drop(cols_with_missing_more_than_75_percent,axis=1)
predicts= pipeline.predict(X_test)
sub_df=pd.DataFrame({'Id':X_test.Id,'SalePrice':predicts})
sub_df.to_csv('../output/xgboost_submission.csv',index=False)